In [69]:
import pandas as pd
df = pd.read_excel('Listed Companies at PSX - HK - 25AUG2022.xlsx')


In [70]:
df = df[df['Shariah Status'] == 'Compliant']
list_of_companies = list(df['Company Symbol'])


In [72]:
# list_of_companies

In [1]:
final_df = pd.DataFrame()

In [80]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
import warnings
warnings.filterwarnings('ignore')

list_of_id = ['gbox_listisq4','gbox_listisq3','gbox_listisq2','gbox_listisq1']

# Initialize a Selenium webdriver instance (e.g., Chrome)
driver = webdriver.Chrome()
for company_symbol in list_of_companies:
    
    url = 'https://www.scstrade.com/stockscreening/SS_CompanySnapShotQF.aspx?symbol=' + company_symbol
    driver.get(url)
    for id_name in list_of_id:
        try:
            # Wait for the page to load (you might need to adjust the waiting time)
            driver.implicitly_wait(10)

            # Locate the div element by its ID
            div_element = driver.find_element_by_id(id_name)

            # Get the HTML content of the div element
            div_html = div_element.get_attribute('innerHTML')

            # Use BeautifulSoup to parse the HTML content of the div
            soup = BeautifulSoup(div_html, 'html.parser')

            # Find and extract the table rows within the div
            table_rows = soup.find_all('tr')

            # Create a list to store the table data
            data = []
            for row in table_rows:
                # Extract and split the text into columns
                columns = [col.get_text(strip=True) for col in row.find_all(['td', 'th'])]
                data.append(columns)
            sleep(3)
            # Create a DataFrame from the table data
            df = pd.DataFrame(data=data[2:],columns=data[0])
            df.set_index("Year / Quarter", inplace=True)

            # Transpose the DataFrame to make "Year / Quarter" values the column names
            df = df.transpose()

            filtered_df = df[df.index.str.contains('Q')]
            filtered_df.replace(to_replace='0',value=np.nan,inplace=True)
            filtered_df["Company Symbol"] = company_symbol
            filtered_df.dropna(inplace=True)
            
            final_df = pd.concat([filtered_df,final_df])
        except:
            continue
    # Close the browser window

driver.quit()


In [81]:
final_df

Year / Quarter,Sales,Cost Of Sales,Gross Profit,Operating Profit,Other Income,Finance Cost,Profit Before Tax,Taxation,Profit After Tax,Company Symbol,Net Interest Income
2023 / Q1,"240,339","179,544","60,795","24,910",-229,"18,437","6,702","2,697","4,005",BNWM,NaN
2022 / Q1,"292,302","214,003","78,299","47,230","-1,885","9,079","37,106","8,678","28,428",BNWM,NaN
2023 / Q2,"621,899","456,073","165,826","128,312","1,327","15,763","97,458","49,562","47,896",BNWM,NaN
2022 / Q2,"560,935","422,015","138,920","97,623","4,151","9,038","72,183","24,777","47,406",BNWM,NaN
2023 / Q3,"84,071","83,332",739,"-43,748","1,525","12,326","-49,872","-12,196","-37,676",BNWM,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2023 / Q2,"10,577,842","8,878,368","1,699,474","1,395,921","209,540","156,802","1,319,128","746,507","572,621",AGTL,NaN
2022 / Q2,"11,246,416","8,999,186","2,247,230","1,979,506","18,117","17,468","1,842,623","1,070,851","771,772",AGTL,NaN
2023 / Q3,"11,378,827","9,317,027","2,061,800","1,588,602","62,096","-110,353","1,638,748","657,966","980,782",AGTL,NaN
2022 / Q3,"6,484,210","5,963,664","520,546","334,786","87,918","4,616","377,846","125,009","252,837",AGTL,NaN


In [82]:
final_df = final_df.reset_index().rename(columns={'index': 'year/quarter'})

In [83]:
date_column = final_df.pop('Company Symbol')
final_df.insert(0, 'Company Symbol', date_column)

In [84]:
q = {'Q1':'01-01-','Q2':'01-04-','Q3':'01-07-','Q4':'01-10-'}
q['Q1'] + '2023'
x = list(final_df['year/quarter'])
tmp = []
for i in x:
    z = q[str(i[-2:])] + str(i[:4])
    tmp.append(z)

In [85]:
final_df['year/quarter'] = tmp

In [87]:
final_df.to_excel('data.xlsx')